## How to design a band-pass filter? 
What is important to know when designing a bandpass filter beyond just using some black box scipy.signal method? 

impulse response: response of the filter to a short pulse 
IIR filter: infinite impulse response filter 
FIR filter: finite impulse response filter, computationally a bit more costly 

convolution theorem states: the result of a convolution is a weighted combination of the frequency structure of the data and that of the kernel. 
       —> if the kernel has a certain frequency range that is present and other that are zero, than the zero frequencies will be attenuated 

* in filter design, frequencies are normally represented as fraction of the Nyquist frequency. 
* filter order is the length of the filter kernel (-1). It determines the precision of the filter’s frequency response. 
* lower bound on the order: if I want to resolve a certain frequency in the signal, the length of the filter kernel should be long enough to at least contain one cycle of that frequency. e.g., for 10Hz the filter should at least be 100ms long in terms of samples. 
* in practice, the filter length should be 2 to 5 times the cycle length of the minimal frequency 
* in theory the filter order should be an even number 

The width of the frequency range of the filter, e.g, the plateaux, determines the frequency resolution and the time resolution. The narrower it is, the better the frequency resolution. But a narrow frequency range in the frequency domain requires a wider kernel in the time domain. E. g., a single frequency pulse in the frequency response of the filter would correspond to a perfect sine wave that never dampens. 

Sharp edges in the frequency response of the filter result in ringing artifacts in the filtered time signal. The transition zones of the bandpass filter should therefore be wide enough to avoid ringing. 

by applying the filter convolution once forward and once backward in time, one can avoid the phase delay that is introduced by the causal filter 

The filter can also be designed with the window method. In this case one only specifies the lower and upper bound of the band pass, the filter method automatically chooses 0 transition zones and afterwards smoothes the filter with a certain window function, e.g., the hamming window. 


In [ ]:
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append('../')
import utils as ut

In [ ]:
# load some data for testing
folder = '/Users/Jan/Dropbox/Master/LR_Kuehn/data/dystonia_rest/for_python/good_theta/'
file = 'spmeeg_13.mat'
d = ut.load_data_spm(file, folder)

In [ ]:
lfp = d['data'][1]
fs = d['fsample'][0][0]

## Design a filter with firls method, explicitly

In [ ]:
# the filter order, should 3x the cycle length of the smallest frequency in the desired band 
nyq = fs / 2.
band = [4., 12.]  # band in Hz
# get the number of samples per ms
samples_per_ms = fs / 1000.
# get the cycle length in samples 
cycle_length = int(1000 / band[0] * samples_per_ms)
numtaps = 3 * cycle_length
# make numtaps even 
if not numtaps % 2: 
    numtaps += 1

In [ ]:
# set the filter parameters
transition_start = band[0] - np.floor(0.1 * (band[1] - band[0])) 
# a hack to avoid negative transition start
if transition_start < 1: 
    transition_start = 1.
transition_stop = band[1] + np.ceil(0.1 * (band[1] - band[0]))
bands = (0, transition_start, band[0], band[1], transition_stop, nyq)
# set the desired filter response 
desired = (0, 0, 1, 1, 0, 0)
# design the filter 
coefs = sp.firls(numtaps, bands, desired, nyq=nyq)

## Design it with the window method 

In [ ]:
# design the filter 
coefs_win = sp.firwin(numtaps=numtaps, cutoff=(band[0], band[1]), window='hamming', nyq=nyq, pass_zero=False)
# get the frequency response 
freq_win, response_win = sp.freqz(coefs_win)

## Design IIR filter with butter

In [ ]:
wn = band / nyq
b, a = sp.butter(Wn=wn, N=4, btype='bandpass')
freq_butter, response_butter = sp.freqz(b, a)

In [ ]:
# get the frequency response 
freq, response = sp.freqz(coefs)

# plot only up to a certain frequency for better visu
upto = int(band[1] + 20)

# plot 
plt.figure(figsize=(15, 5)) 
plt.semilogy((nyq*freq/(np.pi))[:upto], np.abs(response)[:upto], label='firs')
plt.semilogy((nyq*freq_win/(np.pi))[:upto], np.abs(response_win)[:upto], label='firwin')
plt.semilogy((nyq*freq_butter/(np.pi))[:upto], np.abs(response_butter)[:upto], label='butter')
plt.semilogy(bands, desired)
plt.xlim([0, upto])
plt.title('Frequency response')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.grid(True)
plt.legend();

## Check goodness of filters 
One can check the goodness of the filter by comparing it to the ideal filter. the ideal filter is just the connection of the filter parameters stated above with straight lines. A measure of comparison is the sum of squares between them. One certainly has to interpolate for the ideal filter to compare to the filter coefficients of the actual filter. 

The SS should close to zero. Larger than 1. is bad. 

In [ ]:
# construct the ideal filter response, i.e., a vector that has 1s in the bandpass and 0s everywhere else
desired = np.zeros_like(response)
# get the frequencies in Hz 
f = nyq*freq/(np.pi)
# construct a mask of the band
mask = ut.get_array_mask(f >= band[0], f <= band[1])
# set values in mask to 1
desired[mask] = 1 

In [ ]:
# calculate sum of squares
ss = np.sum((desired - abs(response))**2)
ss_win = np.sum((desired - abs(response_win))**2)
ss_butter = np.sum((desired - abs(response_butter))**2)
print('firls:', abs(ss))
print('firwin: ', abs(ss_win))
print('butter: ', abs(ss_butter))

## Apply the filters and compare results

In [ ]:
# use forward backward filtering the avoid phase delay
y = sp.filtfilt(coefs, 1, lfp)
y_win = sp.filtfilt(coefs_win, 1, lfp)
y_butter = sp.filtfilt(b, a, lfp)

In [ ]:
# plot the filtered signal and the original 
plt.figure(figsize=(15, 5))
# only a certain range for better visu
start = 1000
stop = start + 1000

plt.plot(y[start:stop], label='firs')
plt.plot(y_win[start:stop], label='firwin')
plt.plot(y_butter[start:stop], label='butter')
plt.plot(lfp[start:stop], label='raw', alpha=.4)
plt.legend();

## Conclusion
Looking at the value of the SS between the desired and the actual response it seems that the firwin method is the best choice because it automatically chooses the transition zones. 

The choice of the transition zones becomes more difficult if the lower bandpass is small and the range is large, etc. 

The butter IIR filter has a bad frequency resolution, e.g., see the green curve in the response plot and the large value in the sum of squares. But this may be due to the fact that it has an infinite impulse response? 